In [ ]:
# Imports
import pandas as pd
import random
import numpy as np
from gurobipy import Model, GRB, quicksum

In [ ]:
# # Function to generate unique random animal names
# def generate_team_names(num_teams):
#     animals = ["Eagles", "Lions", "Wolves", "Dolphins", "Falcons", "Tigers", "Owls", "Hawks", "Cougars", "Sharks", "Panthers", "Bears"]
#     random.shuffle(animals)
#     return animals[:num_teams]

# # Function to create the fantasy football league schedule for the first 11 weeks
# def create_first_11_weeks_schedule(num_teams):
#     team_names = generate_team_names(num_teams)
#     schedule = []

#     # Generate the first 11 weeks where each team plays each other team once
#     for week in range(1, num_teams):
#         matchups = []

#         for i in range(num_teams // 2):
#             home_team = team_names[i]
#             away_team = team_names[num_teams - 1 - i]

#             if week % 2 == 0:  # Switch home and away every week
#                 home_team, away_team = away_team, home_team

#             matchups.append((week, home_team, away_team))

#         schedule.extend(matchups)
#         team_names = [team_names[-1]] + team_names[:-1]  # Rotate teams for the next week

#     return schedule

# # Function to extend the schedule for weeks 12-14
# def extend_schedule_for_weeks_12_14(schedule):
#     # Copy the matchups for weeks 1-3 and switch home and away for weeks 12-14
#     copied_matchups = schedule[:num_teams - 1]
#     copied_matchups = [(week + 11, home_team, away_team) for week, home_team, away_team in copied_matchups]
#     schedule.extend(copied_matchups)

# # Create the fantasy football league schedule for the first 11 weeks
# num_teams = 12
# first_11_weeks_schedule = create_first_11_weeks_schedule(num_teams)

# # Extend the schedule for weeks 12-14
# extend_schedule_for_weeks_12_14(first_11_weeks_schedule)

# # Create a pandas DataFrame with the schedule data
# columns = ['Week', 'Team', 'Opponent']
# season_schedule = pd.DataFrame(first_11_weeks_schedule, columns=columns)

# Read the CSV file into a DataFrame
season_schedule = pd.read_csv('schedule_teams.csv')
season_schedule.sort_values(by=['Week', 'Team'], inplace=True)

# Display the schedule
print(season_schedule)

In [ ]:
# Create a list of the teams
def generate_team_names(num_teams):
    animals = ["Eagles", "Lions", "Wolves", "Dolphins", "Falcons", "Tigers", "Owls", "Hawks", "Cougars", "Sharks", "Panthers", "Bears"]
    random.shuffle(animals)
    return animals[:num_teams]

# Generate schedule for a specific team
def generate_team_schedule(teams, target_team):
    rotating_teams = teams.copy()
    rotating_teams.remove(target_team)
    random.shuffle(rotating_teams)

    schedule = {target_team: {}}
    for week in range(1, 15):  # Assuming a 14-week season
        matchups = list(zip([target_team] + rotating_teams[:len(rotating_teams)//2], rotating_teams[len(rotating_teams)//2:]))
        opponent = None

        for team1, team2 in matchups:
            if team1 == target_team:
                opponent = team2
                break

            if team2 == target_team:
                opponent = team1
                break

        schedule[target_team][week] = opponent
        rotating_teams = rotating_teams[-1:] + rotating_teams[:-1]  # Rotate the teams

    return schedule

# Generate animal team names for 12 teams
teams = generate_team_names(12)

# Choose the team for which you want to generate the schedule (e.g., "Bears")
target_team = "Bears"

# Generate the schedule for the specified team
team_schedule = generate_team_schedule(teams, target_team)

# Print the generated animal team names
print("Animal Team Names:", teams)

# Print the generated schedule for the specified team
print(f"\nGenerated Schedule for {target_team}:")
for week, opponent in team_schedule[target_team].items():
    print(f"Week {week}: {opponent}")


In [ ]:
# Generate draft order
def generate_team_names(num_teams):
    animals = ["Eagles", "Lions", "Wolves", "Dolphins", "Falcons", "Tigers", "Owls", "Hawks", "Cougars", "Sharks", "Panthers", "Bears"]
    random.shuffle(animals)
    return animals[:num_teams]

# Generate animal team names
team_names = generate_team_names(10)

# Set the initial draft order
initial_draft_order = team_names

# Generate the Snake-Draft order for all rounds
draft_order = []
for r in range(16):
    if r % 2 == 0:
        draft_order.append(initial_draft_order)
    else:
        draft_order.append(initial_draft_order[::-1])  # Reverse the order

# Display the generated team names and draft order
print("Team Names:", team_names)
for round_num, order in enumerate(draft_order, 1):
    print(f"Round {round_num} Draft Order:", order)


In [ ]:
# Load prepared data
fantasy_data_player_rf = pd.read_csv('fantasy_data_player_rf.csv')
fantasy_data_player_xgb = pd.read_csv('fantasy_data_player_xgb.csv')
adp_and_vor_ranks = pd.read_csv('adp_and_vor_ranks.csv')
consensus_ranks = pd.read_csv('consensus_ranks.csv')
merged_adp_ranks = pd.read_csv('merged_adp_ranks.csv')
opportunity_cost_avg_ranks = pd.read_csv('opportunity_cost_avg_ranks.csv')
opportunity_cost_ranks = pd.read_csv('opportunity_cost_ranks.csv')

# Adjust sorting of ranks if necessary and keep only necessary columns
adp_and_vor_ranks = adp_and_vor_ranks.sort_values(by=['rank'])
adp_and_vor_ranks = adp_and_vor_ranks[['player_id', 'Player', 'POS', 'position_group', 'adp_rank', 'rank']]
consensus_ranks = consensus_ranks.sort_values(by=['rank'])
consensus_ranks = consensus_ranks[['player_id', 'Player', 'POS', 'position_group', 'rank']]
merged_adp_ranks = merged_adp_ranks.sort_values(by=['rank'])
merged_adp_ranks = merged_adp_ranks[['player_id', 'Player', 'POS', 'position_group', 'rank']]
opportunity_cost_avg_ranks = opportunity_cost_avg_ranks.sort_values(by=['rank'])
opportunity_cost_avg_ranks = opportunity_cost_avg_ranks[['player_id', 'Player', 'POS', 'position_group', 'rank']]
opportunity_cost_ranks = opportunity_cost_ranks.sort_values(by=['rank'])
opportunity_cost_ranks = opportunity_cost_ranks[['player_id', 'Player', 'POS', 'position_group', 'rank']]


In [ ]:
# Set Multi-index for Fantasy Player Data
fantasy_data_player_rf.set_index(['player_id', 'week'], inplace=True)

# Generate Matrix of Expected Fantasy Points
projected_points = fantasy_data_player_rf.pivot_table(values='predicted_fantasy_points', index=['player_id'], columns=['week'], fill_value=0)

# Reset Index for Fantasy Player Data
fantasy_data_player_rf = fantasy_data_player_rf.reset_index()

# Transform to Long-format DataFrame
full_projected_points = projected_points.reset_index().melt(id_vars='player_id', var_name='week', value_name='points')
full_projected_points['week'] = full_projected_points['week'].astype(int)  # Ensure 'week' is an integer

# Create a dictionary from the long-format DataFrame
projected_points_dict = {(row['player_id'], row['week']): row['points'] for index, row in full_projected_points.iterrows()}

# # Display the dictionary or DataFrame
# print(projected_points_dict)


In [ ]:
# List of players, teams, and weeks
players = fantasy_data_player_rf['player_id'].unique().tolist()
teams = season_schedule['Team'].unique().tolist()
weeks = list(range(1, 15))
position = dict(zip(fantasy_data_player_rf['player_id'], fantasy_data_player_rf['position_group']))

# Assuming each team selects 16 players
num_players_per_team = 16
rounds = list(range(1, num_players_per_team + 1))

# Create a dictionary to store the rank for each player over the entire season
season_ranking = {row['player_id']: row['rank'] for index, row in adp_and_vor_ranks.iterrows()}

# Initialize the model
m = Model("Fantasy_Football_Optimization")

# l[p][k][t] is 1, if player p is selected and played in week k by team t
l = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="played")

# s[p][t] is 1, if player p is selected by team t for the entire season
s = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# d[t][r] is 1, if team t selects a player in round r
d = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# w[t][k] is 1, if team t wins in week k
w = m.addVars(teams, weeks, vtype=GRB.BINARY, name="wins")

# Objective function: Maximize the number of wins for Team Bears
m.setObjective(quicksum(w['Bears', k] for k in weeks), GRB.MAXIMIZE)

# Schedule constraints
for k in weeks:
    opponent = team_schedule['Bears'][k]
    m.addConstr(quicksum(l[p, k, 'Bears'] * projected_points_dict[(p, k)] for p in players) -
                quicksum(l[p, k, opponent] * projected_points_dict[(p, k)] for p in players) >=
                w['Bears', k] - (1 - w[opponent, k]))

for t in teams:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for p in players) == 9)

# Add position constraints
for t in teams:
    for k in weeks:
        # Quarterback Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'QB') == 1)

        # Flex Constraint
        m.addConstr(
            quicksum(l[p, k, t] for p in players if position[p] in ['WR', 'RB']) +
            quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 3
        )

        # Running Back Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'RB') >= 2)

        # Wide Receiver Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'WR') >= 2)

        # Tight End Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 1)

        # Kicker Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'K') == 1)

        # Defense/Special Teams Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'DST') == 1)



for p in players:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for t in teams) <= 1)

for p in players:
    for t in teams:
        m.addConstr(s[p, t] >= quicksum(l[p, k, t] for k in weeks) / len(weeks))

for p in players:
    m.addConstr(quicksum(s[p, t] for t in teams) <= 1)

for t in teams:
    if t != 'Bears':
        m.addConstr(quicksum(s[p, t] * season_ranking[p] for p in players if p in season_ranking and (p, t) in s) <= len(players))

for t in teams:
    m.addConstr(quicksum(s[p, t] for p in players) <= num_players_per_team)

for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(d[t, r] == 1)

# Solve the problem
m.optimize()

# Output the results
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

# Extract selected players for each team and week
selected_players_adp_vor = [(p, k, t) for p in players for k in weeks for t in teams if l[p, k, t].x == 1]

# Convert the list to a DataFrame for better presentation
weekly_lineups_adp_vor = pd.DataFrame(selected_players_adp_vor, columns=['Player ID', 'Week', 'Team'])

weekly_lineups_adp_vor


In [ ]:
# Displaying weekly fantasy points scored by different teams
weekly_points = pd.merge(weekly_lineups_adp_vor, fantasy_data_player_rf[['player_id', 'week', 'fantasy_points_ppr']], 
                      left_on=['Player ID', 'Week'], 
                      right_on=['player_id', 'week'], 
                      how='left')

weekly_points = weekly_points.groupby(['Team', 'Week']).agg({'fantasy_points_ppr': 'sum'}).reset_index()

weekly_points = weekly_points.rename(columns={'fantasy_points_ppr': 'Total Points'})

print(weekly_points[weekly_points['Team'] == 'Bears'])

print(weekly_points[weekly_points['Team'] != 'Bears'])


In [ ]:
# Displaying Bears game results with Result column
games_team = pd.concat({k: pd.Series(v) for k, v in team_schedule.items()}).reset_index()
games_team.columns = ['Team', 'Week', 'Opponent']

match_score = pd.merge(weekly_points, games_team, how='left', on=['Team', 'Week'])

match_score['Opponent Points'] = match_score.apply(
    lambda row: weekly_points[
        (weekly_points['Team'] == row['Opponent']) &
        (weekly_points['Week'] == row['Week'])
    ]['Total Points'].values[0]
    if row['Opponent'] in weekly_points['Team'].values else 0,
    axis=1
)

match_score['Result'] = np.where(match_score['Total Points'] > match_score['Opponent Points'], 'Win', 'Loss')

team_results = match_score[match_score['Team'] == 'Bears']

print(team_results[['Week', 'Team', 'Total Points', 'Opponent', 'Opponent Points', 'Result']])


In [ ]:
# List of players, teams, and weeks
players = fantasy_data_player_rf['player_id'].unique().tolist()
teams = season_schedule['Team'].unique().tolist()
weeks = list(range(1, 15))
position = dict(zip(fantasy_data_player_rf['player_id'], fantasy_data_player_rf['position_group']))

# Assuming each team selects 16 players
num_players_per_team = 16
rounds = list(range(1, num_players_per_team + 1))

# Create a dictionary to store the rank for each player over the entire season
season_ranking = {row['player_id']: row['rank'] for index, row in merged_adp_ranks.iterrows()}

# Initialize the model
m = Model("Fantasy_Football_Optimization")

# l[p][k][t] is 1, if player p is selected and played in week k by team t
l = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="chosen")

# s[p][t] is 1, if player p is selected by team t for the entire season
s = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# d[t][r] is 1, if team t selects a player in round r
d = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# w[t][k] is 1, if team t wins in week k
w = m.addVars(teams, weeks, vtype=GRB.BINARY, name="wins")

# Objective function: Maximize the number of wins for Manager_1
m.setObjective(quicksum(w['Bears', k] for k in weeks), GRB.MAXIMIZE)

# Schedule constraints
for k in weeks:
    opponent = team_schedule['Bears'][k]
    m.addConstr(quicksum(l[p, k, 'Bears'] * projected_points_dict[(p, k)] for p in players) -
                quicksum(l[p, k, opponent] * projected_points_dict[(p, k)] for p in players) >=
                w['Bears', k] - (1 - w[opponent, k]))

for t in teams:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for p in players) == 9)

# Add position constraints
for t in teams:
    for k in weeks:
        # Quarterback Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'QB') == 1)

        # Flex Constraint
        m.addConstr(
            quicksum(l[p, k, t] for p in players if position[p] in ['WR', 'RB']) +
            quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 3
        )

        # Running Back Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'RB') >= 2)

        # Wide Receiver Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'WR') >= 2)

        # Tight End Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 1)

        # Kicker Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'K') == 1)

        # Defense/Special Teams Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'DST') == 1)

for p in players:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for t in teams) <= 1)

for p in players:
    for t in teams:
        m.addConstr(s[p, t] >= quicksum(l[p, k, t] for k in weeks) / len(weeks))

for p in players:
    m.addConstr(quicksum(s[p, t] for t in teams) <= 1)

for t in teams:
    if t != 'Bears':
        m.addConstr(quicksum(s[p, t] * season_ranking[p] for p in players if p in season_ranking and (p, t) in s) <= len(players))

for t in teams:
    m.addConstr(quicksum(s[p, t] for p in players) <= num_players_per_team)

for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(d[t, r] == 1)

# Solve the problem
m.optimize()

# Output the results
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

# Extract selected players for each team and week
selected_players_merged_adp = [(p, k, t) for p in players for k in weeks for t in teams if l[p, k, t].x == 1]

# Convert the list to a DataFrame for better presentation
weekly_lineups_merged_adp = pd.DataFrame(selected_players_merged_adp, columns=['Player ID', 'Week', 'Team'])

weekly_lineups_merged_adp


In [ ]:
# Displaying weekly fantasy points scored by different managers
weekly_points = pd.merge(weekly_lineups_merged_adp, fantasy_data_player_rf[['player_id', 'week', 'fantasy_points_ppr']], 
                      left_on=['Player ID', 'Week'], 
                      right_on=['player_id', 'week'], 
                      how='left')

weekly_points = weekly_points.groupby(['Team', 'Week']).agg({'fantasy_points_ppr': 'sum'}).reset_index()

weekly_points = weekly_points.rename(columns={'fantasy_points_ppr': 'Total Points'})

# Displaying Bears game results with Result column
games_team = pd.concat({k: pd.Series(v) for k, v in team_schedule.items()}).reset_index()
games_team.columns = ['Team', 'Week', 'Opponent']

match_score = pd.merge(weekly_points, games_team, how='left', on=['Team', 'Week'])

match_score['Opponent Points'] = match_score.apply(
    lambda row: weekly_points[
        (weekly_points['Team'] == row['Opponent']) &
        (weekly_points['Week'] == row['Week'])
    ]['Total Points'].values[0]
    if row['Opponent'] in weekly_points['Team'].values else 0,
    axis=1
)

match_score['Result'] = np.where(match_score['Total Points'] > match_score['Opponent Points'], 'Win', 'Loss')

team_results = match_score[match_score['Team'] == 'Bears']

print(team_results[['Week', 'Team', 'Total Points', 'Opponent', 'Opponent Points', 'Result']])

In [ ]:
# List of players, teams, and weeks
players = fantasy_data_player_rf['player_id'].unique().tolist()
teams = season_schedule['Team'].unique().tolist()
weeks = list(range(1, 15))
position = dict(zip(fantasy_data_player_rf['player_id'], fantasy_data_player_rf['position_group']))

# Assuming each team selects 16 players
num_players_per_team = 16
rounds = list(range(1, num_players_per_team + 1))

# Create a dictionary to store the rank for each player over the entire season
season_ranking = {row['player_id']: row['rank'] for index, row in consensus_ranks.iterrows()}

# Initialize the model
m = Model("Fantasy_Football_Optimization")

# l[p][k][t] is 1, if player p is selected and played in week k by team t
l = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="chosen")

# s[p][t] is 1, if player p is selected by team t for the entire season
s = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# d[t][r] is 1, if team t selects a player in round r
d = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# w[t][k] is 1, if team t wins in week k
w = m.addVars(teams, weeks, vtype=GRB.BINARY, name="wins")

# Objective function: Maximize the number of wins for Manager_1
m.setObjective(quicksum(w['Bears', k] for k in weeks), GRB.MAXIMIZE)

# Schedule constraints
for k in weeks:
    opponent = team_schedule['Bears'][k]
    m.addConstr(quicksum(l[p, k, 'Bears'] * projected_points_dict[(p, k)] for p in players) -
                quicksum(l[p, k, opponent] * projected_points_dict[(p, k)] for p in players) >=
                w['Bears', k] - (1 - w[opponent, k]))

for t in teams:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for p in players) == 9)

# Add position constraints
for t in teams:
    for k in weeks:
        # Quarterback Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'QB') == 1)

        # Flex Constraint
        m.addConstr(
            quicksum(l[p, k, t] for p in players if position[p] in ['WR', 'RB']) +
            quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 3
        )

        # Running Back Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'RB') >= 2)

        # Wide Receiver Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'WR') >= 2)

        # Tight End Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 1)

        # Kicker Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'K') == 1)

        # Defense/Special Teams Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'DST') == 1)

for p in players:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for t in teams) <= 1)

for p in players:
    for t in teams:
        m.addConstr(s[p, t] >= quicksum(l[p, k, t] for k in weeks) / len(weeks))

for p in players:
    m.addConstr(quicksum(s[p, t] for t in teams) <= 1)

for t in teams:
    if t != 'Bears':
        m.addConstr(quicksum(s[p, t] * season_ranking[p] for p in players if p in season_ranking and (p, t) in s) <= len(players))

for t in teams:
    m.addConstr(quicksum(s[p, t] for p in players) <= num_players_per_team)

for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(d[t, r] == 1)

# Solve the problem
m.optimize()

# Output the results
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

# Extract selected players for each team and week
selected_players_consensus = [(p, k, t) for p in players for k in weeks for t in teams if l[p, k, t].x == 1]

# Convert the list to a DataFrame for better presentation
weekly_lineups_consensus = pd.DataFrame(selected_players_consensus, columns=['Player ID', 'Week', 'Team'])

weekly_lineups_consensus


In [ ]:
# Displaying weekly fantasy points scored by different managers
weekly_points = pd.merge(weekly_lineups_consensus, fantasy_data_player_rf[['player_id', 'week', 'fantasy_points_ppr']], 
                      left_on=['Player ID', 'Week'], 
                      right_on=['player_id', 'week'], 
                      how='left')

weekly_points = weekly_points.groupby(['Team', 'Week']).agg({'fantasy_points_ppr': 'sum'}).reset_index()

weekly_points = weekly_points.rename(columns={'fantasy_points_ppr': 'Total Points'})

# Displaying Bears game results with Result column
games_team = pd.concat({k: pd.Series(v) for k, v in team_schedule.items()}).reset_index()
games_team.columns = ['Team', 'Week', 'Opponent']

match_score = pd.merge(weekly_points, games_team, how='left', on=['Team', 'Week'])

match_score['Opponent Points'] = match_score.apply(
    lambda row: weekly_points[
        (weekly_points['Team'] == row['Opponent']) &
        (weekly_points['Week'] == row['Week'])
    ]['Total Points'].values[0]
    if row['Opponent'] in weekly_points['Team'].values else 0,
    axis=1
)

match_score['Result'] = np.where(match_score['Total Points'] > match_score['Opponent Points'], 'Win', 'Loss')

team_results = match_score[match_score['Team'] == 'Bears']

print(team_results[['Week', 'Team', 'Total Points', 'Opponent', 'Opponent Points', 'Result']])

In [ ]:
# List of players, teams, and weeks
players = fantasy_data_player_rf['player_id'].unique().tolist()
teams = season_schedule['Team'].unique().tolist()
weeks = list(range(1, 15))
position = dict(zip(fantasy_data_player_rf['player_id'], fantasy_data_player_rf['position_group']))

# Assuming each team selects 16 players
num_players_per_team = 16
rounds = list(range(1, num_players_per_team + 1))

# Create a dictionary to store the rank for each player over the entire season
season_ranking = {row['player_id']: row['rank'] for index, row in opportunity_cost_avg_ranks.iterrows()}

# Initialize the model
m = Model("Fantasy_Football_Optimization")

# l[p][k][t] is 1, if player p is selected and played in week k by team t
l = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="chosen")

# s[p][t] is 1, if player p is selected by team t for the entire season
s = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# d[t][r] is 1, if team t selects a player in round r
d = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# w[t][k] is 1, if team t wins in week k
w = m.addVars(teams, weeks, vtype=GRB.BINARY, name="wins")

# Objective function: Maximize the number of wins for Manager_1
m.setObjective(quicksum(w['Bears', k] for k in weeks), GRB.MAXIMIZE)

# Schedule constraints
for k in weeks:
    opponent = team_schedule['Bears'][k]
    m.addConstr(quicksum(l[p, k, 'Bears'] * projected_points_dict[(p, k)] for p in players) -
                quicksum(l[p, k, opponent] * projected_points_dict[(p, k)] for p in players) >=
                w['Bears', k] - (1 - w[opponent, k]))

for t in teams:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for p in players) == 9)

# Add position constraints
for t in teams:
    for k in weeks:
        # Quarterback Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'QB') == 1)

        # Flex Constraint
        m.addConstr(
            quicksum(l[p, k, t] for p in players if position[p] in ['WR', 'RB']) +
            quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 3
        )

        # Running Back Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'RB') >= 2)

        # Wide Receiver Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'WR') >= 2)

        # Tight End Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 1)

        # Kicker Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'K') == 1)

        # Defense/Special Teams Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'DST') == 1)

for p in players:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for t in teams) <= 1)

for p in players:
    for t in teams:
        m.addConstr(s[p, t] >= quicksum(l[p, k, t] for k in weeks) / len(weeks))

for p in players:
    m.addConstr(quicksum(s[p, t] for t in teams) <= 1)

for t in teams:
    if t != 'Bears':
        m.addConstr(quicksum(s[p, t] * season_ranking[p] for p in players if p in season_ranking and (p, t) in s) <= len(players))

for t in teams:
    m.addConstr(quicksum(s[p, t] for p in players) <= num_players_per_team)

for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(d[t, r] == 1)

# Solve the problem
m.optimize()

# Output the results
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

# Extract selected players for each team and week
selected_players_opp_avg = [(p, k, t) for p in players for k in weeks for t in teams if l[p, k, t].x == 1]

# Convert the list to a DataFrame for better presentation
weekly_lineups_opp_avg = pd.DataFrame(selected_players_opp_avg, columns=['Player ID', 'Week', 'Team'])

weekly_lineups_opp_avg


In [ ]:
# Displaying weekly fantasy points scored by different managers
weekly_points = pd.merge(weekly_lineups_opp_avg, fantasy_data_player_rf[['player_id', 'week', 'fantasy_points_ppr']], 
                      left_on=['Player ID', 'Week'], 
                      right_on=['player_id', 'week'], 
                      how='left')

weekly_points = weekly_points.groupby(['Team', 'Week']).agg({'fantasy_points_ppr': 'sum'}).reset_index()

weekly_points = weekly_points.rename(columns={'fantasy_points_ppr': 'Total Points'})

# Displaying Bears game results with Result column
games_team = pd.concat({k: pd.Series(v) for k, v in team_schedule.items()}).reset_index()
games_team.columns = ['Team', 'Week', 'Opponent']

match_score = pd.merge(weekly_points, games_team, how='left', on=['Team', 'Week'])

match_score['Opponent Points'] = match_score.apply(
    lambda row: weekly_points[
        (weekly_points['Team'] == row['Opponent']) &
        (weekly_points['Week'] == row['Week'])
    ]['Total Points'].values[0]
    if row['Opponent'] in weekly_points['Team'].values else 0,
    axis=1
)

match_score['Result'] = np.where(match_score['Total Points'] > match_score['Opponent Points'], 'Win', 'Loss')

team_results = match_score[match_score['Team'] == 'Bears']

print(team_results[['Week', 'Team', 'Total Points', 'Opponent', 'Opponent Points', 'Result']])

In [ ]:
# List of players, teams, and weeks
players = fantasy_data_player_rf['player_id'].unique().tolist()
teams = season_schedule['Team'].unique().tolist()
weeks = list(range(1, 15))
position = dict(zip(fantasy_data_player_rf['player_id'], fantasy_data_player_rf['position_group']))

# Assuming each team selects 16 players
num_players_per_team = 16
rounds = list(range(1, num_players_per_team + 1))

# Create a dictionary to store the rank for each player over the entire season
season_ranking = {row['player_id']: row['rank'] for index, row in opportunity_cost_ranks.iterrows()}

# Initialize the model
m = Model("Fantasy_Football_Optimization")

# p[i][k][t] is 1, if player i is selected and played in week k by team t
l = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="chosen")

# s[i][t] is 1, if player i is selected by team t for the entire season
s = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# d[t][r] is 1, if team t selects a player in round r
d = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# w[t][k] is 1, if team t wins in week k
w = m.addVars(teams, weeks, vtype=GRB.BINARY, name="wins")

# Objective function: Maximize the number of wins for Manager_1
m.setObjective(quicksum(w['Bears', k] for k in weeks), GRB.MAXIMIZE)

# Schedule constraints
for k in weeks:
    opponent = team_schedule['Bears'][k]
    m.addConstr(quicksum(l[p, k, 'Bears'] * projected_points_dict[(p, k)] for p in players) -
                quicksum(l[p, k, opponent] * projected_points_dict[(p, k)] for p in players) >=
                w['Bears', k] - (1 - w[opponent, k]))

for t in teams:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for p in players) == 9)

# Add position constraints
for t in teams:
    for k in weeks:
        # Quarterback Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'QB') == 1)

        # Flex Constraint
        m.addConstr(
            quicksum(l[p, k, t] for p in players if position[p] in ['WR', 'RB']) +
            quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 3
        )

        # Running Back Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'RB') >= 2)

        # Wide Receiver Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'WR') >= 2)

        # Tight End Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 1)

        # Kicker Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'K') == 1)

        # Defense/Special Teams Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'DST') == 1)

for p in players:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for t in teams) <= 1)

for p in players:
    for t in teams:
        m.addConstr(s[p, t] >= quicksum(l[p, k, t] for k in weeks) / len(weeks))

for p in players:
    m.addConstr(quicksum(s[p, t] for t in teams) <= 1)

for t in teams:
    if t != 'Bears':
        m.addConstr(quicksum(s[p, t] * season_ranking[p] for p in players if p in season_ranking and (p, t) in s) <= len(players))

for t in teams:
    m.addConstr(quicksum(s[p, t] for p in players) <= num_players_per_team)

for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(d[t, r] == 1)

# Solve the problem
m.optimize()

# Output the results
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

# Extract selected players for each team and week
selected_players_opp = [(p, k, t) for p in players for k in weeks for t in teams if l[p, k, t].x == 1]

# Convert the list to a DataFrame for better presentation
weekly_lineups_opp = pd.DataFrame(selected_players_opp, columns=['Player ID', 'Week', 'Team'])

weekly_lineups_opp


In [ ]:
# Displaying weekly fantasy points scored by different managers
weekly_points = pd.merge(weekly_lineups_opp, fantasy_data_player_rf[['player_id', 'week', 'fantasy_points_ppr']], 
                      left_on=['Player ID', 'Week'], 
                      right_on=['player_id', 'week'], 
                      how='left')

weekly_points = weekly_points.groupby(['Team', 'Week']).agg({'fantasy_points_ppr': 'sum'}).reset_index()

weekly_points = weekly_points.rename(columns={'fantasy_points_ppr': 'Total Points'})

# Displaying Bears game results with Result column
games_team = pd.concat({k: pd.Series(v) for k, v in team_schedule.items()}).reset_index()
games_team.columns = ['Team', 'Week', 'Opponent']

match_score = pd.merge(weekly_points, games_team, how='left', on=['Team', 'Week'])

match_score['Opponent Points'] = match_score.apply(
    lambda row: weekly_points[
        (weekly_points['Team'] == row['Opponent']) &
        (weekly_points['Week'] == row['Week'])
    ]['Total Points'].values[0]
    if row['Opponent'] in weekly_points['Team'].values else 0,
    axis=1
)

match_score['Result'] = np.where(match_score['Total Points'] > match_score['Opponent Points'], 'Win', 'Loss')

team_results = match_score[match_score['Team'] == 'Bears']

print(team_results[['Week', 'Team', 'Total Points', 'Opponent', 'Opponent Points', 'Result']])

In [ ]:
# Set Multi-index for Fantasy Player Data
fantasy_data_player_xgb.set_index(['player_id', 'week'], inplace=True)

# Generate Matrix of Expected Fantasy Points
projected_points_xgb = fantasy_data_player_xgb.pivot_table(values='predicted_fantasy_points', index=['player_id'], columns=['week'], fill_value=0)

# Reset Index for Fantasy Player Data
fantasy_data_player_xgb = fantasy_data_player_xgb.reset_index()

# Transform to Long-format DataFrame
full_projected_points_xgb = projected_points_xgb.reset_index().melt(id_vars='player_id', var_name='week', value_name='points')
full_projected_points_xgb['week'] = full_projected_points_xgb['week'].astype(int)  # Ensure 'week' is an integer

# Create a dictionary from the long-format DataFrame
projected_points_dict_xgb = {(row['player_id'], row['week']): row['points'] for index, row in full_projected_points_xgb.iterrows()}

# # Display the dictionary or DataFrame
# print(projected_points_dict_xgb)


In [ ]:
# List of players, teams, and weeks
players = fantasy_data_player_xgb['player_id'].unique().tolist()
teams = season_schedule['Team'].unique().tolist()
weeks = list(range(1, 15))
position = dict(zip(fantasy_data_player_xgb['player_id'], fantasy_data_player_xgb['position_group']))

# Assuming each team selects 16 players
num_players_per_team = 16
rounds = list(range(1, num_players_per_team + 1))

# Create a dictionary to store the rank for each player over the entire season
season_ranking = {row['player_id']: row['rank'] for index, row in adp_and_vor_ranks.iterrows()}

# Initialize the model
m = Model("Fantasy_Football_Optimization")

# l[p][k][t] is 1, if player p is selected and played in week k by team t
l = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="played")

# s[p][t] is 1, if player p is selected by team t for the entire season
s = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# d[t][r] is 1, if team t selects a player in round r
d = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# w[t][k] is 1, if team t wins in week k
w = m.addVars(teams, weeks, vtype=GRB.BINARY, name="wins")

# Objective function: Maximize the number of wins for Team Bears
m.setObjective(quicksum(w['Bears', k] for k in weeks), GRB.MAXIMIZE)

# Schedule constraints
for k in weeks:
    opponent = team_schedule['Bears'][k]
    m.addConstr(quicksum(l[p, k, 'Bears'] * projected_points_dict_xgb[(p, k)] for p in players) -
                quicksum(l[p, k, opponent] * projected_points_dict_xgb[(p, k)] for p in players) >=
                w['Bears', k] - (1 - w[opponent, k]))

for t in teams:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for p in players) == 9)

# Add position constraints
for t in teams:
    for k in weeks:
        # Quarterback Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'QB') == 1)

        # Flex Constraint
        m.addConstr(
            quicksum(l[p, k, t] for p in players if position[p] in ['WR', 'RB']) +
            quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 3
        )

        # Running Back Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'RB') >= 2)

        # Wide Receiver Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'WR') >= 2)

        # Tight End Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 1)

        # Kicker Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'K') == 1)

        # Defense/Special Teams Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'DST') == 1)

for p in players:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for t in teams) <= 1)

for p in players:
    for t in teams:
        m.addConstr(s[p, t] >= quicksum(l[p, k, t] for k in weeks) / len(weeks))

for p in players:
    m.addConstr(quicksum(s[p, t] for t in teams) <= 1)

for t in teams:
    if t != 'Bears':
        m.addConstr(quicksum(s[p, t] * season_ranking[p] for p in players if p in season_ranking and (p, t) in s) <= len(players))

for t in teams:
    m.addConstr(quicksum(s[p, t] for p in players) <= num_players_per_team)

for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(d[t, r] == 1)

# Solve the problem
m.optimize()

# Output the results
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

# Extract selected players for each team and week
selected_players_adp_vor = [(p, k, t) for p in players for k in weeks for t in teams if l[p, k, t].x == 1]

# Convert the list to a DataFrame for better presentation
weekly_lineups_adp_vor = pd.DataFrame(selected_players_adp_vor, columns=['Player ID', 'Week', 'Team'])

weekly_lineups_adp_vor

# Displaying weekly fantasy points scored by different managers
weekly_points = pd.merge(weekly_lineups_adp_vor, fantasy_data_player_xgb[['player_id', 'week', 'fantasy_points_ppr']], 
                      left_on=['Player ID', 'Week'], 
                      right_on=['player_id', 'week'], 
                      how='left')

weekly_points = weekly_points.groupby(['Team', 'Week']).agg({'fantasy_points_ppr': 'sum'}).reset_index()

weekly_points = weekly_points.rename(columns={'fantasy_points_ppr': 'Total Points'})

# Displaying Bears game results with Result column
games_team = pd.concat({k: pd.Series(v) for k, v in team_schedule.items()}).reset_index()
games_team.columns = ['Team', 'Week', 'Opponent']

match_score = pd.merge(weekly_points, games_team, how='left', on=['Team', 'Week'])

match_score['Opponent Points'] = match_score.apply(
    lambda row: weekly_points[
        (weekly_points['Team'] == row['Opponent']) &
        (weekly_points['Week'] == row['Week'])
    ]['Total Points'].values[0]
    if row['Opponent'] in weekly_points['Team'].values else 0,
    axis=1
)

match_score['Result'] = np.where(match_score['Total Points'] > match_score['Opponent Points'], 'Win', 'Loss')

team_results = match_score[match_score['Team'] == 'Bears']

print(team_results[['Week', 'Team', 'Total Points', 'Opponent', 'Opponent Points', 'Result']])


In [ ]:
# List of players, teams, and weeks
players = fantasy_data_player_xgb['player_id'].unique().tolist()
teams = season_schedule['Team'].unique().tolist()
weeks = list(range(1, 15))
position = dict(zip(fantasy_data_player_xgb['player_id'], fantasy_data_player_xgb['position_group']))

# Assuming each team selects 16 players
num_players_per_team = 16
rounds = list(range(1, num_players_per_team + 1))

# Create a dictionary to store the rank for each player over the entire season
season_ranking = {row['player_id']: row['rank'] for index, row in merged_adp_ranks.iterrows()}

# Initialize the model
m = Model("Fantasy_Football_Optimization")

# l[p][k][t] is 1, if player p is selected and played in week k by team t
l = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="played")

# s[p][t] is 1, if player p is selected by team t for the entire season
s = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# d[t][r] is 1, if team t selects a player in round r
d = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# w[t][k] is 1, if team t wins in week k
w = m.addVars(teams, weeks, vtype=GRB.BINARY, name="wins")

# Objective function: Maximize the number of wins for Team Bears
m.setObjective(quicksum(w['Bears', k] for k in weeks), GRB.MAXIMIZE)

# Schedule constraints
for k in weeks:
    opponent = team_schedule['Bears'][k]
    m.addConstr(quicksum(l[p, k, 'Bears'] * projected_points_dict_xgb[(p, k)] for p in players) -
                quicksum(l[p, k, opponent] * projected_points_dict_xgb[(p, k)] for p in players) >=
                w['Bears', k] - (1 - w[opponent, k]))

for t in teams:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for p in players) == 9)

# Add position constraints
for t in teams:
    for k in weeks:
        # Quarterback Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'QB') == 1)

        # Flex Constraint
        m.addConstr(
            quicksum(l[p, k, t] for p in players if position[p] in ['WR', 'RB']) +
            quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 3
        )

        # Running Back Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'RB') >= 2)

        # Wide Receiver Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'WR') >= 2)

        # Tight End Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 1)

        # Kicker Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'K') == 1)

        # Defense/Special Teams Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'DST') == 1)

for p in players:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for t in teams) <= 1)

for p in players:
    for t in teams:
        m.addConstr(s[p, t] >= quicksum(l[p, k, t] for k in weeks) / len(weeks))

for p in players:
    m.addConstr(quicksum(s[p, t] for t in teams) <= 1)

for t in teams:
    if t != 'Bears':
        m.addConstr(quicksum(s[p, t] * season_ranking[p] for p in players if p in season_ranking and (p, t) in s) <= len(players))

for t in teams:
    m.addConstr(quicksum(s[p, t] for p in players) <= num_players_per_team)

for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(d[t, r] == 1)

# Solve the problem
m.optimize()

# Output the results
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

# Extract selected players for each team and week
selected_players_merged_adp = [(p, k, t) for p in players for k in weeks for t in teams if l[p, k, t].x == 1]

# Convert the list to a DataFrame for better presentation
weekly_lineups_merged_adp = pd.DataFrame(selected_players_merged_adp, columns=['Player ID', 'Week', 'Team'])

weekly_lineups_merged_adp

# Displaying weekly fantasy points scored by different managers
weekly_points = pd.merge(weekly_lineups_merged_adp, fantasy_data_player_xgb[['player_id', 'week', 'fantasy_points_ppr']], 
                      left_on=['Player ID', 'Week'], 
                      right_on=['player_id', 'week'], 
                      how='left')

weekly_points = weekly_points.groupby(['Team', 'Week']).agg({'fantasy_points_ppr': 'sum'}).reset_index()

weekly_points = weekly_points.rename(columns={'fantasy_points_ppr': 'Total Points'})

# Displaying Bears game results with Result column
games_team = pd.concat({k: pd.Series(v) for k, v in team_schedule.items()}).reset_index()
games_team.columns = ['Team', 'Week', 'Opponent']

match_score = pd.merge(weekly_points, games_team, how='left', on=['Team', 'Week'])

match_score['Opponent Points'] = match_score.apply(
    lambda row: weekly_points[
        (weekly_points['Team'] == row['Opponent']) &
        (weekly_points['Week'] == row['Week'])
    ]['Total Points'].values[0]
    if row['Opponent'] in weekly_points['Team'].values else 0,
    axis=1
)

match_score['Result'] = np.where(match_score['Total Points'] > match_score['Opponent Points'], 'Win', 'Loss')

team_results = match_score[match_score['Team'] == 'Bears']

print(team_results[['Week', 'Team', 'Total Points', 'Opponent', 'Opponent Points', 'Result']])

In [ ]:
# List of players, teams, and weeks
players = fantasy_data_player_xgb['player_id'].unique().tolist()
teams = season_schedule['Team'].unique().tolist()
weeks = list(range(1, 15))
position = dict(zip(fantasy_data_player_xgb['player_id'], fantasy_data_player_xgb['position_group']))

# Assuming each team selects 16 players
num_players_per_team = 16
rounds = list(range(1, num_players_per_team + 1))

# Create a dictionary to store the rank for each player over the entire season
season_ranking = {row['player_id']: row['rank'] for index, row in consensus_ranks.iterrows()}

# Initialize the model
m = Model("Fantasy_Football_Optimization")

# l[p][k][t] is 1, if player p is selected and played in week k by team t
l = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="played")

# s[p][t] is 1, if player p is selected by team t for the entire season
s = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# d[t][r] is 1, if team t selects a player in round r
d = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# w[t][k] is 1, if team t wins in week k
w = m.addVars(teams, weeks, vtype=GRB.BINARY, name="wins")

# Objective function: Maximize the number of wins for Team Bears
m.setObjective(quicksum(w['Bears', k] for k in weeks), GRB.MAXIMIZE)

# Schedule constraints
for k in weeks:
    opponent = team_schedule['Bears'][k]
    m.addConstr(quicksum(l[p, k, 'Bears'] * projected_points_dict_xgb[(p, k)] for p in players) -
                quicksum(l[p, k, opponent] * projected_points_dict_xgb[(p, k)] for p in players) >=
                w['Bears', k] - (1 - w[opponent, k]))

for t in teams:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for p in players) == 9)

# Add position constraints
for t in teams:
    for k in weeks:
        # Quarterback Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'QB') == 1)

        # Flex Constraint
        m.addConstr(
            quicksum(l[p, k, t] for p in players if position[p] in ['WR', 'RB']) +
            quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 3
        )

        # Running Back Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'RB') >= 2)

        # Wide Receiver Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'WR') >= 2)

        # Tight End Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 1)

        # Kicker Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'K') == 1)

        # Defense/Special Teams Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'DST') == 1)

for p in players:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for t in teams) <= 1)

for p in players:
    for t in teams:
        m.addConstr(s[p, t] >= quicksum(l[p, k, t] for k in weeks) / len(weeks))

for p in players:
    m.addConstr(quicksum(s[p, t] for t in teams) <= 1)

for t in teams:
    if t != 'Bears':
        m.addConstr(quicksum(s[p, t] * season_ranking[p] for p in players if p in season_ranking and (p, t) in s) <= len(players))

for t in teams:
    m.addConstr(quicksum(s[p, t] for p in players) <= num_players_per_team)

for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(d[t, r] == 1)

# Solve the problem
m.optimize()

# Output the results
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

# Extract selected players for each team and week
selected_players_consensus = [(p, k, t) for p in players for k in weeks for t in teams if l[p, k, t].x == 1]

# Convert the list to a DataFrame for better presentation
weekly_lineups_consensus = pd.DataFrame(selected_players_consensus, columns=['Player ID', 'Week', 'Team'])

weekly_lineups_consensus

# Displaying weekly fantasy points scored by different managers
weekly_points = pd.merge(weekly_lineups_consensus, fantasy_data_player_xgb[['player_id', 'week', 'fantasy_points_ppr']], 
                      left_on=['Player ID', 'Week'], 
                      right_on=['player_id', 'week'], 
                      how='left')

weekly_points = weekly_points.groupby(['Team', 'Week']).agg({'fantasy_points_ppr': 'sum'}).reset_index()

weekly_points = weekly_points.rename(columns={'fantasy_points_ppr': 'Total Points'})

# Displaying Bears game results with Result column
games_team = pd.concat({k: pd.Series(v) for k, v in team_schedule.items()}).reset_index()
games_team.columns = ['Team', 'Week', 'Opponent']

match_score = pd.merge(weekly_points, games_team, how='left', on=['Team', 'Week'])

match_score['Opponent Points'] = match_score.apply(
    lambda row: weekly_points[
        (weekly_points['Team'] == row['Opponent']) &
        (weekly_points['Week'] == row['Week'])
    ]['Total Points'].values[0]
    if row['Opponent'] in weekly_points['Team'].values else 0,
    axis=1
)

match_score['Result'] = np.where(match_score['Total Points'] > match_score['Opponent Points'], 'Win', 'Loss')

team_results = match_score[match_score['Team'] == 'Bears']

print(team_results[['Week', 'Team', 'Total Points', 'Opponent', 'Opponent Points', 'Result']])

In [ ]:
# List of players, teams, and weeks
players = fantasy_data_player_xgb['player_id'].unique().tolist()
teams = season_schedule['Team'].unique().tolist()
weeks = list(range(1, 15))
position = dict(zip(fantasy_data_player_xgb['player_id'], fantasy_data_player_xgb['position_group']))

# Assuming each team selects 16 players
num_players_per_team = 16
rounds = list(range(1, num_players_per_team + 1))

# Create a dictionary to store the rank for each player over the entire season
season_ranking = {row['player_id']: row['rank'] for index, row in opportunity_cost_avg_ranks.iterrows()}

# Initialize the model
m = Model("Fantasy_Football_Optimization")

# l[p][k][t] is 1, if player p is selected and played in week k by team t
l = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="played")

# s[p][t] is 1, if player p is selected by team t for the entire season
s = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# d[t][r] is 1, if team t selects a player in round r
d = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# w[t][k] is 1, if team t wins in week k
w = m.addVars(teams, weeks, vtype=GRB.BINARY, name="wins")

# Objective function: Maximize the number of wins for Team Bears
m.setObjective(quicksum(w['Bears', k] for k in weeks), GRB.MAXIMIZE)

# Schedule constraints
for k in weeks:
    opponent = team_schedule['Bears'][k]
    m.addConstr(quicksum(l[p, k, 'Bears'] * projected_points_dict_xgb[(p, k)] for p in players) -
                quicksum(l[p, k, opponent] * projected_points_dict_xgb[(p, k)] for p in players) >=
                w['Bears', k] - (1 - w[opponent, k]))

for t in teams:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for p in players) == 9)

# Add position constraints
for t in teams:
    for k in weeks:
        # Quarterback Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'QB') == 1)

        # Flex Constraint
        m.addConstr(
            quicksum(l[p, k, t] for p in players if position[p] in ['WR', 'RB']) +
            quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 3
        )

        # Running Back Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'RB') >= 2)

        # Wide Receiver Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'WR') >= 2)

        # Tight End Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 1)

        # Kicker Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'K') == 1)

        # Defense/Special Teams Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'DST') == 1)

for p in players:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for t in teams) <= 1)

for p in players:
    for t in teams:
        m.addConstr(s[p, t] >= quicksum(l[p, k, t] for k in weeks) / len(weeks))

for p in players:
    m.addConstr(quicksum(s[p, t] for t in teams) <= 1)

for t in teams:
    if t != 'Bears':
        m.addConstr(quicksum(s[p, t] * season_ranking[p] for p in players if p in season_ranking and (p, t) in s) <= len(players))

for t in teams:
    m.addConstr(quicksum(s[p, t] for p in players) <= num_players_per_team)

for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(d[t, r] == 1)

# Solve the problem
m.optimize()

# Output the results
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

# Extract selected players for each team and week
selected_players_opp_avg = [(p, k, t) for p in players for k in weeks for t in teams if l[p, k, t].x == 1]

# Convert the list to a DataFrame for better presentation
weekly_lineups_opp_avg = pd.DataFrame(selected_players_opp_avg, columns=['Player ID', 'Week', 'Team'])

weekly_lineups_opp_avg

# Displaying weekly fantasy points scored by different managers
weekly_points = pd.merge(weekly_lineups_opp_avg, fantasy_data_player_xgb[['player_id', 'week', 'fantasy_points_ppr']], 
                      left_on=['Player ID', 'Week'], 
                      right_on=['player_id', 'week'], 
                      how='left')

weekly_points = weekly_points.groupby(['Team', 'Week']).agg({'fantasy_points_ppr': 'sum'}).reset_index()

weekly_points = weekly_points.rename(columns={'fantasy_points_ppr': 'Total Points'})

# Displaying Bears game results with Result column
games_team = pd.concat({k: pd.Series(v) for k, v in team_schedule.items()}).reset_index()
games_team.columns = ['Team', 'Week', 'Opponent']

match_score = pd.merge(weekly_points, games_team, how='left', on=['Team', 'Week'])

match_score['Opponent Points'] = match_score.apply(
    lambda row: weekly_points[
        (weekly_points['Team'] == row['Opponent']) &
        (weekly_points['Week'] == row['Week'])
    ]['Total Points'].values[0]
    if row['Opponent'] in weekly_points['Team'].values else 0,
    axis=1
)

match_score['Result'] = np.where(match_score['Total Points'] > match_score['Opponent Points'], 'Win', 'Loss')

team_results = match_score[match_score['Team'] == 'Bears']

print(team_results[['Week', 'Team', 'Total Points', 'Opponent', 'Opponent Points', 'Result']])

In [ ]:
# List of players, teams, and weeks
players = fantasy_data_player_xgb['player_id'].unique().tolist()
teams = season_schedule['Team'].unique().tolist()
weeks = list(range(1, 15))
position = dict(zip(fantasy_data_player_xgb['player_id'], fantasy_data_player_xgb['position_group']))

# Assuming each team selects 16 players
num_players_per_team = 16
rounds = list(range(1, num_players_per_team + 1))

# Create a dictionary to store the rank for each player over the entire season
season_ranking = {row['player_id']: row['rank'] for index, row in opportunity_cost_ranks.iterrows()}

# Initialize the model
m = Model("Fantasy_Football_Optimization")

# l[p][k][t] is 1, if player p is selected and played in week k by team t
l = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="played")

# s[p][t] is 1, if player p is selected by team t for the entire season
s = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# d[t][r] is 1, if team t selects a player in round r
d = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# w[t][k] is 1, if team t wins in week k
w = m.addVars(teams, weeks, vtype=GRB.BINARY, name="wins")

# Objective function: Maximize the number of wins for Team Bears
m.setObjective(quicksum(w['Bears', k] for k in weeks), GRB.MAXIMIZE)

# Schedule constraints
for k in weeks:
    opponent = team_schedule['Bears'][k]
    m.addConstr(quicksum(l[p, k, 'Bears'] * projected_points_dict_xgb[(p, k)] for p in players) -
                quicksum(l[p, k, opponent] * projected_points_dict_xgb[(p, k)] for p in players) >=
                w['Bears', k] - (1 - w[opponent, k]))

for t in teams:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for p in players) == 9)

# Add position constraints
for t in teams:
    for k in weeks:
        # Quarterback Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'QB') == 1)

        # Flex Constraint
        m.addConstr(
            quicksum(l[p, k, t] for p in players if position[p] in ['WR', 'RB']) +
            quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 3
        )

        # Running Back Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'RB') >= 2)

        # Wide Receiver Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'WR') >= 2)

        # Tight End Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 1)

        # Kicker Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'K') == 1)

        # Defense/Special Teams Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'DST') == 1)

for p in players:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for t in teams) <= 1)

for p in players:
    for t in teams:
        m.addConstr(s[p, t] >= quicksum(l[p, k, t] for k in weeks) / len(weeks))

for p in players:
    m.addConstr(quicksum(s[p, t] for t in teams) <= 1)

for t in teams:
    if t != 'Bears':
        m.addConstr(quicksum(s[p, t] * season_ranking[p] for p in players if p in season_ranking and (p, t) in s) <= len(players))

for t in teams:
    m.addConstr(quicksum(s[p, t] for p in players) <= num_players_per_team)

for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(d[t, r] == 1)

# Solve the problem
m.optimize()

# Output the results
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

# Extract selected players for each team and week
selected_players_opp = [(p, k, t) for p in players for k in weeks for t in teams if l[p, k, t].x == 1]

# Convert the list to a DataFrame for better presentation
weekly_lineups_opp = pd.DataFrame(selected_players_opp, columns=['Player ID', 'Week', 'Team'])

weekly_lineups_opp

# Displaying weekly fantasy points scored by different managers
weekly_points = pd.merge(weekly_lineups_opp, fantasy_data_player_xgb[['player_id', 'week', 'fantasy_points_ppr']], 
                      left_on=['Player ID', 'Week'], 
                      right_on=['player_id', 'week'], 
                      how='left')

weekly_points = weekly_points.groupby(['Team', 'Week']).agg({'fantasy_points_ppr': 'sum'}).reset_index()

weekly_points = weekly_points.rename(columns={'fantasy_points_ppr': 'Total Points'})

# Displaying Bears game results with Result column
games_team = pd.concat({k: pd.Series(v) for k, v in team_schedule.items()}).reset_index()
games_team.columns = ['Team', 'Week', 'Opponent']

match_score = pd.merge(weekly_points, games_team, how='left', on=['Team', 'Week'])

match_score['Opponent Points'] = match_score.apply(
    lambda row: weekly_points[
        (weekly_points['Team'] == row['Opponent']) &
        (weekly_points['Week'] == row['Week'])
    ]['Total Points'].values[0]
    if row['Opponent'] in weekly_points['Team'].values else 0,
    axis=1
)

match_score['Result'] = np.where(match_score['Total Points'] > match_score['Opponent Points'], 'Win', 'Loss')

team_results = match_score[match_score['Team'] == 'Bears']

print(team_results[['Week', 'Team', 'Total Points', 'Opponent', 'Opponent Points', 'Result']])